https://nijianmo.github.io/amazon/index.html

In [ ]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Grocery_and_Gourmet_Food_5.json.gz

--2022-03-27 02:05:26--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Grocery_and_Gourmet_Food_5.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 146631394 (140M) [application/octet-stream]
Saving to: ‘Grocery_and_Gourmet_Food_5.json.gz’

Grocery_and_Gourmet 100%[===================>] 139.84M  42.0MB/s    in 3.7s    

2022-03-27 02:05:30 (37.9 MB/s) - ‘Grocery_and_Gourmet_Food_5.json.gz’ saved [146631394/146631394]



In [ ]:
def parse(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield json.loads(l)

In [ ]:
import pandas as pd
import gzip
import json

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('Grocery_and_Gourmet_Food_5.json.gz')

In [ ]:
ratings = []

for review in parse("Grocery_and_Gourmet_Food_5.json.gz"):
  ratings.append(review['overall'])

print( sum(ratings) / len(ratings) )

4.442208836745755


In [ ]:
print(df.columns)

#drop na
df1 = df.dropna()


Index(['overall', 'verified', 'reviewTime', 'reviewerID', 'asin',
       'reviewerName', 'reviewText', 'summary', 'unixReviewTime', 'vote',
       'style', 'image'],
      dtype='object')


In [ ]:
#df.reviewText df.asin df.vote df.overall
df2 = df1[ ['asin', 'reviewText', 'vote', 'overall']]
df2.shape
df_groupedby = df2['overall'].groupby(df2['asin'])
product = pd.DataFrame(df_groupedby.mean())
product


overall
asin               
B00006FWVX      4.0
B00008RCN8      5.0
B0000CD02T      3.0
B0000CFH7B      4.6
B0000CFPI2      5.0
...             ...
B01HFK90JE      5.0
B01HH79XRE      5.0
B01HH7IWXU      5.0
B01HH9IN64      5.0
B01HHXJEIG      5.0

[1964 rows x 1 columns]

In [ ]:
!pip install nltk
import pandas , nltk
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import RegexpTokenizer
def nltk_tokenizer(_wd):
  return RegexpTokenizer(r'\w+').tokenize(_wd.lower())

df2['Token_review'] = df2['reviewText'].apply(nltk_tokenizer)
df2.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


asin                                         reviewText vote  \
393  B00006FWVX  I love using these colors to dye my white cand...   14   
578  B00008RCN8  The best price you could ever get, Fresh made ...    4   
873  B0000CD02T  Usually a big fan of Bob's Red Mill products, ...    2   
961  B0000CFH7B  I am writing this based on my use with FROSTIN...   13   
978  B0000CFH7B  Really cool spray.  Be sure that whatever your...    7   

     overall                                       Token_review  
393      4.0  [i, love, using, these, colors, to, dye, my, w...  
578      5.0  [the, best, price, you, could, ever, get, fres...  
873      3.0  [usually, a, big, fan, of, bob, s, red, mill, ...  
961      5.0  [i, am, writing, this, based, on, my, use, wit...  
978      4.0  [really, cool, spray, be, sure, that, whatever...

In [ ]:
doc_df = df2[['asin','Token_review']].values.tolist()
tagged_data = [TaggedDocument(words=_d, tags=[uid]) for uid, _d in doc_df]

In [ ]:
len(tagged_data)

3057

In [ ]:
max_epochs = 10

model = Doc2Vec(
    window=10,
    size=100,
    alpha=0.025, 
    min_alpha=0.025,
    min_count=2,
    dm =1,
    negative = 5,
    seed = 9999)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

/usr/local/lib/python3.7/dist-packages/gensim/models/doc2vec.py:570: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


iteration 0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9


In [ ]:
model.random.seed(9999)

doc_list = 'Rooms were clean'.split(' ')

inferred_vector = model.infer_vector(doc_list)
return_docs = model.docvecs.most_similar(positive=[inferred_vector],topn=5)
for rd in return_docs:
  for des in df[df['asin'] == rd[0]]['reviewText']:
    print( rd[0],rd[1],des)

In [ ]:
return_docs = model.docvecs.most_similar('B00006FWVX',topn=5)
for rd in return_docs:
  for des in df[df['asin'] == rd[0]]['asin']:
    print(rd[0],rd[1],des)

In [ ]:
len(model.docvecs.offset2doctag)

1964

In [ ]:
# store the model to mmap-able files
model.save('/tmp/my_model.doc2vec')
# load the model back
model_loaded = Doc2Vec.load('/tmp/my_model.doc2vec')

In [ ]:
#get embedding , merge with vote
#새 문서의 docvec 추정
#newvec = model.infer_vector(['the', 'cat', 'jumped'])

#model.docvecs[1963]
df1 = pd.DataFrame(model.docvecs.vectors_docs)
df2 = pd.DataFrame(model.docvecs.offset2doctag, columns=['id'])
embedding = pd.concat([df2,df1], axis=1)
embedding.columns

Index(['id',    0,    1,    2,    3,    4,    5,    6,    7,    8,
       ...
         90,   91,   92,   93,   94,   95,   96,   97,   98,   99],
      dtype='object', length=101)

In [ ]:
#aggregate df2 by asin 
#merge product, embedding
product['id'] = product.index
embedding2 = pd.merge( product, embedding, on="id")

In [ ]:
#simple dnn for predicting overall with tensorflow
#standardize y
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD 
from tensorflow.keras.losses import mse 

#(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
#print(x_train.shape, y_train.shape,x_test.shape,y_test.shape)

y = embedding2.overall.values
x = embedding2.values[:, 3:103]

#scaling
y = y/5.0

x = x.astype(np.float)

# 파티셔닝: 7:3
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y, test_size = 0.2, random_state =1234) 


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


In [ ]:
#모형 구성
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
model = Sequential()
model.add(Dense(64, activation = 'relu', input_shape = (100, ) ) )
model.add(Dense(32, activation = 'relu')) #32개의 출력을 가지는 Dense층
model.add(Dense(16, activation = 'relu')) #32개의 출력을 가지는 Dense층
model.add(Dense(1)) # 10개의 출력을 가지는 신경망

# 모형 컴파일
model.compile(optimizer = 'adam', loss = 'mse',  metrics=['mae', 'mse'])
# 학습
history = model.fit(x_train, y_train, epochs = 30,batch_size = 128, validation_split=0.2)
history.history.keys()

Epoch 1/30
10/10 [==============================] - 1s 38ms/step - loss: 1.8671 - mae: 1.0760 - mse: 1.8671 - val_loss: 0.5732 - val_mae: 0.5802 - val_mse: 0.5732
Epoch 2/30
10/10 [==============================] - 0s 10ms/step - loss: 0.3879 - mae: 0.4747 - mse: 0.3879 - val_loss: 0.4907 - val_mae: 0.5070 - val_mse: 0.4907
Epoch 3/30
10/10 [==============================] - 0s 10ms/step - loss: 0.2563 - mae: 0.4018 - mse: 0.2563 - val_loss: 0.3019 - val_mae: 0.4300 - val_mse: 0.3019
Epoch 4/30
10/10 [==============================] - 0s 10ms/step - loss: 0.2028 - mae: 0.3760 - mse: 0.2028 - val_loss: 0.2629 - val_mae: 0.4058 - val_mse: 0.2629
Epoch 5/30
10/10 [==============================] - 0s 11ms/step - loss: 0.1680 - mae: 0.3424 - mse: 0.1680 - val_loss: 0.2454 - val_mae: 0.3786 - val_mse: 0.2454
Epoch 6/30
10/10 [==============================] - 0s 9ms/step - loss: 0.1443 - mae: 0.3133 - mse: 0.1443 - val_loss: 0.2265 - val_mae: 0.3627 - val_mse: 0.2265
Epoch 7/30
10/10 [=====

dict_keys(['loss', 'mae', 'mse', 'val_loss', 'val_mae', 'val_mse'])

In [ ]:
# 모형 평가-train, test
model.evaluate(x_test, y_test)

#분류 보고서
from sklearn.metrics import classification_report, confusion_matrix
results = model.predict(x_test)
np.sqrt( np.mean(( results-y_test)**2 ) )

#print('\n', classification_report(np.argmax(y_test, axis = -1), np.argmax(results, axis = -1)))

13/13 [==============================] - 0s 2ms/step - loss: 0.1064 - mae: 0.2661 - mse: 0.1064


0.3892813784878636

In [ ]:
#학습결과 확인
import matplotlib.pyplot as plt
his_dict = history.history
loss = his_dict['loss']
val_loss = his_dict['val_loss'] 
acc = his_dict['mse']
val_acc = his_dict['val_mse']
epochs = range(1,11 ) #1~20 epoch

#plot
fig = plt.figure(figsize= (10,5) )

# loss
ax1 = fig.add_subplot(1,2,1)
ax1.plot(epochs, loss, color = 'red', label = 'train')
ax1.plot(epochs, val_loss, color = 'blue', label = 'validation')
ax1.set_title('loss')
ax1.set_xlabel('epochs')
ax1.set_ylabel('loss')
ax1.legend()

#accuracy
ax2 = fig.add_subplot(1,2,2)
ax2.plot(epochs, acc, color = 'red', label = 'train')
ax2.plot(epochs, val_acc, color = 'blue', label = 'validation')
ax2.set_title('accuracy')
ax2.set_xlabel('epochs')
ax2.set_ylabel('accuracy')
ax2.legend()

plt.show()

In [ ]:
#다른 참고 코드
#embedding with doc2vec gensim
#https://radimrehurek.com/gensim/models/doc2vec.html

from sklearn.cluster import KMeans
# common_text에는 파싱된 워드 리스트들이 들어가 있음.
from gensim.test.utils import common_texts

# Doc2Vec이 우리가 텍스트를 사용해서 학습되는 모델(뉴럴넷)이고 
# TaggedDocument가 넘겨주는 텍스트들. 
# 여기서, corpus와 ID들을 함께 넘겨줘야 하는데, 여기서 ID는 tag와 같은 말임
# Q: 여기서, 여러 tag를 함께 넘겨준 다음, 적합한 태그를 찾아주는 방식으로도 처리할 수 있는지 파악하는 것이 필요함. 
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# TaggedDocument는 해당 corpus의 id를 함께 넘겨주는 것을 말합니다. 
# 아래 코드에서 doc에는 단어의 묶음이, tags에는 해당 문서를 표현하는 태그가 들어가게 됨. 
# 흠, 그렇다면, 이 태그에 고유 id가 아닌 다른 것을 넣어주면 현재 문서에 맞는 tag를 찾아주기도 하나? 
# 이렇게 할 경우, words에는 문서를, tags에는 키워드를 넣고, words에서 tags를 예측하는 짓을 할 수도 있지 않을까? 
common_texts_and_tags = [
    (text, [f"str_{i}",]) for i, text in enumerate(common_texts)
]
print("##"*20)
print("tags and its texts")
print("##"*20)
for text, tags in common_texts_and_tags:
    print(f"tags: {tags}, text: {text}")

# text: 단어별로 분할되어 있는 리스트 
# tags: 해당 문서를 의미하는 태그. 여기서, tags를 unique id로 넣기는 했는데, 그렇지 않게 넣어서 학습시키는 것도 가능할 것으로 보임. 
TRAIN_documents = [TaggedDocument(words=text, tags=tags) for text, tags in common_texts_and_tags]
# training. 
# 여러 Parameter들을 사용하여 튜닝할 수 있음. 
model = Doc2Vec(TRAIN_documents, vector_size=5, window=3, epochs=40, min_count=0, workers=4)
print("##"*20)
print("Accessing Document vector")
print("##"*20)
# document의 vector를 구하는 방법은, 기 학습된 docvec을 가져오거나, 해당 단어를 넣고 다시 예측하는 것. 
# 단, tag로 접근이 가능할 뿐, 학습한 doc를 그대로 가져오는 것은 어려운 것으로 보임. 
# 또한, 여기서, 학습된 vector와 inferring한 vector가 미묘하게 다른데, 이것은 현재 train set이 적어서 그런 것으로 보임. 
for text, tags in common_texts_and_tags:
    trained_doc_vec = model.docvecs[tags[0]]
    inferred_doc_vec = model.infer_vector(text)
    print(f"tags: {tags}, text: {text}")
    print(f"trained_doc_vec: {trained_doc_vec}")
    print(f"inferred_doc_vec: {inferred_doc_vec}")
    print("--"*20)
    
print("##"*20)
print("predicting Document vector")
print("##"*20)

new_documents = [
    ["computer", "interface"], 
    ["I", "am", "a", "boy"]
]
# predict training set with its infering vector)
for text in new_documents:
    inferred_v = model.infer_vector(text)
    # 현재 doc를 모델을 사용하여 벡터화할때의 값 
    print(f"vector of {text}: {inferred_v}")
    # 기학습된 문서중에서 현재 벡터와 가장 유사한 벡터를 가지는 문서를 topn만큼 추출합니다. 
    most_similar_docs = model.docvecs.most_similar([inferred_v], topn=3)
    # index와 그 유사도를 함께 보여줍니다. 
    # index(tag)가 아닌 문서를 바로 보여주기는 어려운 것 같고, 
    for index, similarity in most_similar_docs:
        print(f"{index}, similarity: {similarity}")
    #print(most_similar_docs)
    print("=="*20)

# document clustering 

print("##"*30)
print("K-Means Clustering")
print("##"*30)
Clustering_Method = KMeans(n_clusters=2, random_state=0)
X = model.docvecs.vectors_docs # document vector 전체를 가져옴. 
Clustering_Method.fit(X)# fitting 
# 결과를 보면 알겠지만, 생각보다 클러스터링이 잘 되지 않음. 
# 일단은 이것 또한 트레이닝 셋이 적어서 그런 것으로 보임. 
cluster_dict = {i:[] for i in range(0, 2)}
for text_tags, label in zip(common_texts_and_tags, Clustering_Method.labels_):
    text, tags = text_tags
    cluster_dict[label].append(text)
for label, lst in cluster_dict.items():
    print(f"Cluster {label}")
    for x in lst:
        print(x)
    print("--"*30)
print("##"*20)